In [1]:
import nest_asyncio
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI

from src.database import VectorDB, ExtractDocs
from src.model import *
from src.utils import check_required_env_vars

nest_asyncio.apply()
load_dotenv()

/Users/ifish/Documents/Github/louis.ai/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [ ]:
from datasets import load_dataset

corpus = load_dataset('isaacus/open-australian-legal-qa', split='train')

In [ ]:
check_required_env_vars()

In [2]:
db = VectorDB()
# db.enable_hnsw_indexing()

In [ ]:
extract = ExtractDocs()

In [ ]:
import uuid

In [ ]:
text_corpus = [f'{source["citation"]} -> {source["text"]}' for source in corpus["source"]]
metadata = [
    {
        'id': str(uuid.uuid4()),
        'version_id': source["version_id"],
        'type' : source["type"],
        'jurisdiction' : source["jurisdiction"],
        'source' : source["source"],
        'citation' : source["citation"],
        'url' : source["url"],
    } for source in corpus["source"]]

In [ ]:
embedded_corpus = extract.extract_text(text_corpus, metadata)

In [ ]:
db.add_documents(embedded_corpus)

In [ ]:
db.enable_hnsw_indexing()

query = "In the case of R v NGUYEN [2001] NSWCCA 334, what was the relationship between the Appellant and Mr Nguyen, and what activities of Mr Nguyen did the Appellant testify about?"

1. what is the intent of this query
2. am I fit to answer this question
3. does the intent includes wanting to analyse a document
4. summarise the document if document is longer than x and add to user_context in query
5. do I need more system_context in answering this question
6. which part of the intent am I missing that requires more system_context
7. how can I search the vector store for the missing system_context
8. is my system_context enough (go back to 3 if no)
9. let's put the information together and generate an answer
10. is there hallucination to my answer? (go back to 3 if no) 
11. is there biasness (go back to 2 if yes)
12. is my answer relevant to my question (go back to 4 if no)
13. does the intent includes wanting to generate a document

In [3]:
# query = """Tell me about the Liberty of the person under the constitution of Singapore:
# (5)  Clauses (3) and (4) shall not apply to an enemy alien or to any person arrested for 
# # contempt of Parliament pursuant to a warrant issued under the hand of the Speaker."""
query = "In the case of R v NGUYEN [2001] NSWCCA 334, what was the relationship between the Appellant and Mr Nguyen, and what activities of Mr Nguyen did the Appellant testify about?"
# query = "Can I count to the number 30 with someone else's hands?"
# query = "Is it legal to call someone a 'dumb dumb bubblegum'?"
# query = "人を殺すってだめですか？"
# query = "Would it be illegal if I went 'bippity boppity, I'mma steal your intellectual property' in Singapore?"
# query = "Is it illegal to be stupid in Singapore?"
# query = "What is the weather today"
# 

In [ ]:
vectorstore_summary = "It includes the reliable information of the case laws in Australia."

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

inputs = {
    "query": query,
    "db": db,
    "model": model,
    "vectorstore_summary": vectorstore_summary,
    "retrieved_docs":[""],
    "depth": 0,
    "excluded_file_ids": set(),
    "intent_type": "qa",
    "user_context": "",
}

response = build_graph()
app = response.compile()

output = app.invoke(inputs)

ValueError: Invalid input type <class 'langchain_core.messages.human.HumanMessage'>. Must be a PromptValue, str, or list of BaseMessages.

In [ ]:
from pprint import pprint

pprint(output)

In [6]:
print(output["response"].content)

**Legal Basis**:  
In the case of *R v NGUYEN [2001] NSWCCA 334*, the legal context involves the testimony of the Appellant regarding her relationship with Mr. Nguyen and his activities. The case is situated within the framework of criminal law, particularly concerning the possession and distribution of illegal substances, which is governed by the relevant statutes in New South Wales, Australia.

**Analysis**:  
The Appellant testified that Mr. Nguyen was her cousin, and she allowed him to live in her flat for about 4 or 5 days before the 6th of November because he had nowhere else to stay. The Appellant was aware of Mr. Nguyen's involvement in selling heroin prior to his moving in, as she had heard about it. On the first day of his stay, she witnessed Mr. Nguyen handing over a small foil to a third person, which she found surprising, upsetting, and angering. Despite her protests and warnings that he would have to move out if he continued such activities, Mr. Nguyen persisted in sellin

In [ ]:
"""
In the case of R v NGUYEN [2001] NSWCCA 334, the Appellant testified that Mr Nguyen was her cousin and that she had allowed him to live in her flat for about 4 or 5 days. She stated that she had heard that Mr Nguyen was selling heroin and that she had seen him hand over a small foil to a third person, an event that made her feel surprised, upset, and angry. Despite her protests, Mr Nguyen allegedly continued to sell heroin from the flat. The Appellant also mentioned seeing other customers in the flat and a friend of Mr Nguyen's cutting foil in the lounge-room. Despite her complaints to her boyfriend and an aunt, she took no further steps to prevent these activities, citing reasons such as their close familial relationship and her reluctance to involve the police.
"""